# Jetpack Compose - Pager + TabRow

Zobaczmy jak w `Jatpack Compose` można wykonać nawigację zakładkową - podobnie jak w przykładzie 3.3. Taką nawigację można wykonać za pomocą `Scaffold` i `topBar`, podobnie jak widzieliśmy w poprzednich przykładach. Tutaj zobaczymy jak wykorzystać `Pager` z `TabRow` aby osiągnąć podobny efekt. Podobnie jak w przykładzie 5.3, musimy odpowwiedni skonfigurować projekt.

<img src="https://media0.giphy.com/media/v1.Y2lkPTc5MGI3NjExNTYwOGVkNmEwNDIyYTU1NTEzYzEyOGEwZmY1ZjQ0NjBmNWFhMDk0ZiZlcD12MV9pbnRlcm5hbF9naWZzX2dpZklkJmN0PWc/99ggiI0zbSvtZSX9AS/giphy.gif" width="200" />

Do `build.gradle (Project)` dodajemy

In [ ]:
buildscript {
    ext {
        compose_ui_version = '1.4.0'
    }
}

Definiujemy właściwości rozszerzeń (*extensions*) dla skryptu budowania, interesuje nas zmienna `compose_ui_version`, która przechowuje numer wersji `Compose UI` używanej w aplikacji - tutaj użyjemy wersji `1.4.0`.

Słowo kluczowe `ext` służy do definiowania właściwości rozszerzeń (*extensions*) dla skryptu budowania, które mogą być wykorzystywane w innych częściach skryptu, np. w konfiguracji zależności (*dependencies*).

W pliku `build.gradle (Module)` dodajjemy zależność, oraz ustawiamy odpowidnie wersje kompilatora

In [ ]:
android {
    ...
    composeOptions {
        kotlinCompilerExtensionVersion '1.4.3'
    }
    ...
}

dependencies {

    implementation 'androidx.core:core-ktx:1.8.0'
    implementation 'androidx.lifecycle:lifecycle-runtime-ktx:2.3.1'
    implementation 'androidx.activity:activity-compose:1.5.1'
    implementation 'androidx.compose.foundation:foundation:1.4.3'
    ...
}

- `kotlinCompilerExtensionVersion` - określa wersję rozszerzenia kompilatora języka Kotlin do Compose.
- `androidx.compose.foundation:foundation` - biblioteka zawierająca podstawowe elementy interfejsu użytkownika w Compose - tutaj potrzebujemy wersję `1.4.3`.

Rozpocznijmy od funkcji `Composable` reprezentującej cały layout

In [ ]:
@OptIn(ExperimentalFoundationApi::class)
@Composable
fun MainLayout(){
    val data = listOf(
        "Tab 1" to Icons.Filled.Home,
        "Tab 2" to Icons.Filled.Person,
        "Tab 3" to Icons.Filled.Phone,
        "Tab 4" to Icons.Filled.Email,
    )

    val pagerState = rememberPagerState()
    val coroutineScope = rememberCoroutineScope()

    Column {
        Tabs(pagerState = pagerState, coroutineScope = coroutineScope, data = data)
        Pages(pagerState = pagerState, data = data)
    }
}

W pierwszej kolejności dodajemy dane - tutaj będzie to lista par (`List<Pair<String, ImageVector>>`) zawierająca nazwy zakładek, wraz z odpowiadającą im ikoną. Następnie dodajemy stan naszego pagera (indeks bieżącej strony oraz liczba dostępnych stron). Ponieważ przez zakładki możemy przechodzić co kilka stron, więc musimy też dodać taką możliwość do naszego pagera. metody `animateScrollToPage` i/lub `scrollToPage` są funkcjami z możliwości zawieszenia wykonania (`suspend fun`), muszą zostać wykonane wewnątrz `Coroutine`. W deklaracji `val coroutineScope = rememberCoroutineScope()`, tworzony jest obiekt `coroutineScope`, który jest używany do zarządzania *coroutines* w Jetpack Compose w asynchroniczny sposób. Pozwala na uruchamianie *coroutines*, czekanie na ich zakończenie i zarządzanie ich cyklem życia.

W kontekście deklaracji `val coroutineScope = rememberCoroutineScope()`, `rememberCoroutineScope()` jest wywoływane raz i wartość zwracana przez tę funkcję jest zapamiętywana przez `Jetpack Compose`, co oznacza, że obiekt `coroutineScope` będzie istniał przez cały cykl życia komponentu. Dzięki temu, korzystając z `coroutineScope`, możemy tworzyć i uruchamiać korutyny wewnątrz komponentu komponującego, co jest szczególnie przydatne w przypadku wykonywania asynchronicznych operacji.

Treść umieszczamy w kolumnie (`Column`) i tutaj wywołujemy dwie funkcje. Funkcja `Tabs` tworzy zakładki, funkcja `Pages` dodaje `HorizontalPager`.

In [ ]:
@OptIn(ExperimentalFoundationApi::class)
@Composable
fun Tabs(pagerState: PagerState, coroutineScope: CoroutineScope, data: List<Pair<String, ImageVector>>){
    TabRow(
        selectedTabIndex = pagerState.currentPage,
    ) {
        data.forEachIndexed { index, pair ->
            Tab(
                selected = pagerState.currentPage == index, 
                onClick = { coroutineScope.launch { pagerState.animateScrollToPage(index)}}, 
                text = { Text(text = pair.first) }, 
                icon = {Icon(imageVector = pair.second, contentDescription = null)}
            )
        }
    }
}

Funkcja jest odpowiedzialna za renderowanie zakładek, która przyjmuje trzy parametry:
- `pagerState` - obiekt `PagerState`, który przechowuje stan paska zakładek i pozwala na nawigację między nimi.
- `coroutineScope` - obiekt `CoroutineScope`, który jest używany do uruchamiania korutyn.
Komponent `TabRow` jest używany do renderowania paska zakładek. Przyjmuje kilka parametrów, tutaj wykorzystamy tylko `selectedTabIndex` - indeks wybranej zakładki połączymy z odpowiednią stroną w pagerze. Wewnątrz `TabRow` iterujemy po elementach listy data przy użyciu funkcji `forEachIndexed`. Iterujemy po elementach wraz z ich indeksami, co pozwala nam na dostęp do indeksu i wartości elementu wewnątrz pętli. Wewnątrz pętli, dla każdej pary, definiujemy pojedynczą zakładkę przy użyciu komponentu `Tab`. Parametry `selected`, ``onClick`, `text` i `icon` są odpowiedzialne za odpowiednie ustawienie stanu, obsługę kliknięcia, tekst i ikonę dla danej zakładki.

In [ ]:
@OptIn(ExperimentalFoundationApi::class)
@Composable
fun Pages(pagerState: PagerState, data: List<Pair<String, ImageVector>>) {
    HorizontalPager(
        state = pagerState,
        modifier = Modifier
            .fillMaxSize()
            .wrapContentSize(Alignment.Center),
        pageCount = data.size,
        pageSize = PageSize.Fill
    ) { index ->
        Column(
            modifier = Modifier.fillMaxSize(),
            verticalArrangement = Arrangement.Center,
            horizontalAlignment = Alignment.CenterHorizontally
        ) {
            Text(
                text = data[index].first,
            )
        }
    }
}

`HorizontalPager` renderuje strony w poziomie. Przyjmuje on kilka parametrów:
- `state` - stan pagera, 
- `modifier` - modyfikator, który kontroluje rozmiar i położenie pagera, 
- `pageCount` - liczba stron
- `pageSize` - rozmiar stron.

Wewnątrz bloku lambda przekazanego do `HorizontalPager`, definiujemy zawartość każdej strony. Blok lambda otrzymuje indeks aktualnie renderowanej strony. Komponent `Column` renderuje zawartość strony w układzie kolumnowym. Przyjmuje on kilka parametrów:

- `modifier` - modyfikator, który kontroluje rozmiar i położenie kolumny, 
- `verticalArrangement` - układ pionowy elementów w kolumnie 
- `horizontalAlignment` wyrównanie poziome elementów w kolumnie.

Wewnątrz `Column` umieszczamy komponent `Text`, który renderuje tekst na stronie. Wyświetlamy tekst pobierając go z pierwszego elementu pary w data dla danego indeksu (`data[index].first`).

Możemy przetestować aplikację

<img src="https://media0.giphy.com/media/v1.Y2lkPTc5MGI3NjExNTYwOGVkNmEwNDIyYTU1NTEzYzEyOGEwZmY1ZjQ0NjBmNWFhMDk0ZiZlcD12MV9pbnRlcm5hbF9naWZzX2dpZklkJmN0PWc/99ggiI0zbSvtZSX9AS/giphy.gif" width="200" />